In this practical session, we explore the use of the Laplacian operator on a surface, in order to solve heat diffusision, to perform shape matching, or even to compute geodesic distances!

In a first step, we learn how to read and store triangulated surfaces (*meshes*), and how to build the discrete Laplace-Beltrami Operator. 

We then seek to use this operator in different practical scenarios.

# Setup envionment 

Your Python environment should include `numpy`, `scipy`, `matplotlib` and `meshplot`. 
The latter can be installed from conda using:
><code>conda install -c conda-forge meshplot
</code>

or with pip with the command:
><code>pip install https://github.com/skoch9/meshplot/archive/0.4.0.tar.gz
>pip install pythreejs
</code>

For unknown reasons, the vizualization does not always work with `jupyter lab`. We recommand using `jupyter notebook`.

Please download [this zip file](https://www.lix.polytechnique.fr/Labo/Robin.Magnet/INF631/data.zip) and place all elements in the same folder as the notebook.

In [1]:
import numpy as np
import scipy.sparse as sparse


# If you did not put the elements of the zip file in the same folder as the the notebook add the next two lines
# import sys
# sys.path.append("PATH TO THE DIRECTORY WHERE YOU UNZIPED YOUR FILES")
import plot_utils as plu # Follow the above procedure

# Reading and storing a mesh 

Unlike images, there are several ways to represent a discrete (triangulated) surface. In this session, we will stick to the most basic data structure which is made of
:
1. A list of vertex coordinates in $\mathbb{R}^3$
2. A list of triangles, each defined by 3 **ordered** vertex indices $i,j,k\in\mathbb{N}$. The order of the vertices define the direction of the normal.

The `OFF` format, or **O**bject **F**ile **F**ormat, stores this information in a `.off` file, which we will use today. 

Open the `bunny.off` file using a text editor, where the structure will look like :
<blockquote>
    <code>OFF
34834 69451 0
-0.0378297 0.12794 0.00447467
-0.0447794 0.128887 0.00190497
...
-0.0400442 0.15362 -0.00816685
3 20463 20462 19669 
3 8845 8935 14299 
...</code>
</blockquote>

As you may already understand, our `.off` file is presented as follows:
<ol>
    <li> First Line: <code>OFF</code> letters</li>
    <li> Second Line:  <code>n m 0</code>, with $n$ the numbers of <b>vertices</b> and $m$ the number of <b>triangles</b></li>
    <li> Next $n$ lines: <code>X Y Z</code> coordinates for each <b>vertex</b></li>
    <li> Next $m$ lines: <code>3 i j k</code> indices of the vertices constituting each <b>face</b></li>
</ol>

In practice, the `OFF` file format can include some more information as you can see on [here](https://en.wikipedia.org/wiki/OFF_(file_format)) if you are interested.

## QUESTION 1
<b> Write the function `read_off` which can read a basic `.off` file.</b>

**Tips** :
<ol>
    <li> You can use the <code>with</code> statement to efficiently process a file: <br>
        <code>with open(filepath, 'r') as f:
   # compute operation on file</code>
    </li>
    <li> <code>f.readline()</code> returns the next line</li>
    <li> <code>text.strip().split()</code> removes breakline characters from <code>text</code> and splits with respect to spaces. Try it with <code>text="1 2 3\n"</code> to see what happens.</li>
</ol>

In [2]:
def read_off(filepath):
    """
    Reads a simple .off file
    
    Input
    --------------
    filepath : str - path to the .off file
    
    Output
    --------------
    vertices : (n,3) array of vertex coordinates (float)
    faces    : (m,3) array of faces defined by vertices index (integers)
    """
    ## TODO
    # READ THE OFF FILE with path "filepath" and return vertex and faces information
    
    with open(filepath, 'r') as f:
        vertices, faces = [] , []
        line1 = f.readline().strip()
        assert line1 == "OFF", f"Erreur : la première ligne est '{line1}', attendu 'OFF'"
        #ligne 2
        n,m,_=f.readline().strip().split()
        n,m=int(n),int(m)
        # vertices
        for _ in range(n):
                vertice=f.readline().strip().split()
                for i in range(3):
                        vertice[i]=float(vertice[i])
                vertices.append(vertice)
        #faces
        for _ in range(m):
                face=f.readline().strip().split()
                face0=[]
                for i in range(1,4):
                        face0.append(int(face[i]))
                faces.append(face0)

        vertices = np.array(vertices, dtype=float)
        faces = np.array(faces, dtype=int) 
    return vertices, faces

As we will have to deal with several components regarding a single mesh, it might be good to create a specific class in which all information will be stored.

In [3]:
class MyMesh:
    def __init__(self, path):
        """
        Initialize the mesh from a path
        """
        self.vertices, self.faces = read_off(path)

This way, if we define an instance of `MyMesh` as `mesh = MyMesh(filepath)`, the `__init__` function is automatically run with parameter `filepath` (ignore the `self` parameter).

Then the mesh vertex coordinates can be accessed using `mesh.vertices`and its faces information with `mesh.faces`. These are the *properties* of our class.

Let us load and visualize our bunny:

In [4]:
mesh = MyMesh('bunny.off')


In [5]:
mesh.vertices.shape, mesh.faces.shape

((34834, 3), (69451, 3))

In [6]:
p=plu.plot(mesh)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

In [7]:
p

# Discrete Operators

As seen in class, the Laplacian of a mesh is ubiquitous in geometry processing. The discrete laplacian is defined as
$$
L = A^{-1}W
$$
with $A$ the (diagonal) area matrix, and $W$ the cotangent weight matrix

**Notes for Sparse Matrices**<br>
As the Laplace-Operator is a differential operator, it only locally acts at a point with respect to a local neighborhood. When storing it in a $n\times n$ matrix, this results in numerous zero entries, which are useless to keep in memory.

For both speed and memory efficiency, we use sparse matrices to represent such operators, where only non-zero entries are stored. The most basic structure for a sparse matrix is the **COO**rdinate format, which consists in three lists $I,J,V$ which respectively store the line index, column index and value for each non-zero entry.

For algebra manipulations (matrix vector multiplications, linear system, ...), other format such as **C**ompressed **S**parse **R**ow (CSR) provide a substential speedup compared to COO. You can read about the CSR and CSC format [here](https://en.wikipedia.org/wiki/Sparse_matrix).

In the [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html) library, you can generate COO, CSR or CSC matrices from the $I,J,V$ format using for instance<br>
> <code>scipy.sparse.csr_matrix((V, (I,J)), shape=(N,N))</code>

Note that depending on your python version, you might now have access to the more recent `scipy.sparse.csr_array` functions.

**Notes for numpy linear algebra**<br>
A few tips for simpler reading of numpy
<ol>
    <li> <code>np.matmul(A, B)</code> and <code>A @ B</code> are the same </li>
    <li><code>np.matmum(A, v)</code> and <code>A @ v</code> are the same</li>
    <li><code>np.dot(u, v)</code> and <code>u @ v</code> are the same</li>
    <li> <code>np.transpose(A)</code> and <code>A.T</code> are the same</li>
</ol>

This means <code>A @ B.T @ u</code> is the same as <code>np.matmul(A, np.matmul(np.transpose(B), u))</code>

## Question 2 
<b> Fill the functions <code>compute_faces_areas</code> and <code>area_matrix</code> to respectively compute the area of each face and the vertex-wise area matrix $A$</b>

Recall the area matrix $A$ is a $n\times n$ diagonal matrix where each entry provide the corresponding vertex area defined as:
$$
A_{ii} = \frac{1}{3}\sum_{f\in\mathcal{F}\\ i\in f} \text{Area}(f)
$$

**Tip**: There are multiple ways to compute the area of a triangle $ABC$, one of them being $\frac{1}{2}\|\vec{AB}\times \vec{AC}\|$

In [8]:
def compute_faces_areas(vertices, faces):
    """
    Compute the area of each face
    
    Input
    --------------
    vertices : (n,3) - vertex coordinates
    faces    : (m,3) - faces defined by vertex indices
    
    Output
    --------------
    faces_areas : (m,) - area of each face
    """
    ## TODO
    # Compute the area of each triangle of the mesh.

    v0 = vertices[faces[:, 0], :] #récupère les coordonnées du premier sommet de chaque face
    v1 = vertices[faces[:, 1], :] #idem pour le deuxième
    v2 = vertices[faces[:, 2], :] #idem pour le troisième

    faces_areas = 0.5 * np.linalg.norm(np.cross(v1 - v0, v2 - v0), axis=1) #calcul de l'aire
    
    return faces_areas

def area_matrix(vertices, faces):
    """
    Compute the diagonal area matrix
    
    Input
    --------------
    vertices : (n,3) - vertex coordinates
    faces    : (m,3) - faces defined by vertex indices
    
    Output
    --------------
    A : (n,n) sparse matrix in DIAgonal format
    """
    #TODO
    # Compute the area of each vertex in the mesh.
    # Use the formula above.
    
    vertex_areas = np.zeros(vertices.shape[0])
    faces_areas = compute_faces_areas(vertices, faces)

    for i in range(len(faces)): #on parcourt toutes les faces, puis les sommets de chaque face auquel on ajoute 1/3 de l'aire de la face
        for j in range(3):
            vertex_areas[faces[i][j]] += faces_areas[i] / 3

    #TODO (takes more than one line)
    
    
    # Create a SPARSE diagonal matix from vertex areas
    N = vertices.shape[0]
    A = sparse.dia_matrix((vertex_areas, 0), shape=(N, N))
    return A


<b>Sanity Check : Verify that your total area remains the same whether you sum all face areas or the entries of the area matrix</b>

In [9]:
faces_areas = compute_faces_areas(mesh.vertices, mesh.faces)

A = area_matrix(mesh.vertices, mesh.faces)
print(A.diagonal().sum() == faces_areas.sum())

True


## Question 3

Recall the stiffness matrix $W$ is a $n\times n$ sparse matrix. There are multiple ways to compute each entry, one of them being:
$$
W_{ij} = \begin{cases}
-\sum_{k\neq i} W_{ik} \quad&\text{if}\quad i=j\\
-\frac{1}{2} \left(\cot(\alpha_{ij}) + \cot(\beta_{ij})\right)\quad&\text{if}\quad ij\in\mathcal{E}\\
0 \quad&\text{else}
\end{cases}
$$

where $\alpha_{ij},\beta_{ij} \in [0,\pi[$ are the unsigned angles opposite to the edge $ij$.

![title](img/cotan_weight.svg)

### Question 3.1 

<b> Fill the function <code>get_cotan_weights</code> to compute the cotangent weights for each face</b>

In [10]:
def cotan_weights(vertices, faces):
    """
    Compute cotangent weights for each face. Each vertex will carry the cotan of its angle.
    
    For a face [i, j, k], the output will be [alpha_{jk}, alpha_{ki}, alpha_{ij}].
    
    Input
    ----------
    vertices : (n,3) - The vertices of the mesh
    faces : (m,3) The faces of the mesh
    
    Output
    -------
    cotan_weights : (m,3) The cotangent weights for each face
    """
    def calcul_angle(A,B,C):
        #Renvoie la valeur de l'angle BAC en A
        AB = B - A
        AC = C - A

        # produit scalaire et normes
        dot = np.dot(AB, AC)
        norm_AB = np.linalg.norm(AB)
        norm_AC = np.linalg.norm(AC)

        cos_theta = dot / (norm_AB * norm_AC)
        cos_theta = np.clip(cos_theta, -1.0, 1.0) # to avoid numerical errors

        theta = np.arccos(cos_theta)
        return theta


    cotan_weights = [np.zeros(3) for _ in range(len(faces))]
    for i in range(len(faces)):
        xi=vertices[faces[i][0],:]
        xj=vertices[faces[i][1],:]
        xk=vertices[faces[i][2],:]

        cotan_weights[i]=np.array([calcul_angle(xi,xj,xk),calcul_angle(xj,xk,xi),calcul_angle(xk,xi,xj)])

    cotan_weights=np.array(cotan_weights)

    return cotan_weights
            
 
    ### TODO - Compute the cotangent weights
    # This can be done in parallel using numpy, or by looping over the faces

### Question 3.2 

<b> Fill the function <code>cotan_matrix</code> to compute the cotangent weight matrix $W$</b>

In [11]:
import scipy
def cotan_matrix(vertices, faces):

    """
    Compute the stiffness matrix
    
    Input
    --------------
    vertices : (n,3) - vertex coordinates
    faces    : (m,3) - faces defined by vertex indices
    
    Output
    --------------
    W : (n,n) sparse matrix in CSC format
    """
    # TODO
    # Compute the entries I,J,V of the stiffness matrix
    # Note that the same pair (i,j) of indices can appear multiple times in I,J
    # The corresponding values in V are then summed by scipy.

    cotan_weights_faces = cotan_weights(vertices, faces)

    #for i!=j, on rempli au fur et à mesure les listes I,J et V
    I=[]
    J=[]
    V=[]
    for i in range(len(faces)):
        weights=cotan_weights_faces[i]
        for j in range(3):
            v1=faces[i][j]
            v2=faces[i][(j+1)%3]
            
            I.append(v1)
            J.append(v2)
            V.append(-0.5/np.tan(weights[(j+2)%3]))
            I.append(v2)
            J.append(v1)
            V.append(-0.5/np.tan(weights[(j+2)%3]))
    
    #pour i=j, on rempli la diagonale en parcourant ce qu'on a déjà calculé. On parcourt toutes les cases avec des valeurs, et on ajoute la valeur opposée au terme diagonal de même premier indice (i,j) sera ajouté à (i,i) et (j,i) à (j,j)
    for i in range(len(I)):
        v1=I[i]
        v2=J[i]
        value=V[i]
        I.append(v1)
        J.append(v1)
        V.append(-value)
    
    #passage sparse csr
    W = scipy.sparse.csr_matrix((V, (I,J)), shape=(len(vertices),len(vertices)))
    
    return W


**Sanity Check : Ensure all terms sum to 0**

In [12]:
np.isclose(cotan_matrix(mesh.vertices, mesh.faces).sum(), 0)

np.True_

Let us now integrate this Laplacian into our mesh class (nothing to change here):

In [13]:
class MyMesh:
    def __init__(self, path):
        """
        Initialize the mesh from a path
        """
        self.vertices, self.faces = read_off(path)
        
    def compute_laplacian(self):
        self.A = area_matrix(self.vertices, self.faces)
        self.W = cotan_matrix(self.vertices, self.faces)

The function `compute_laplacian` is called a *method* of the class `MyMesh`, and can be called as follows:

In [14]:
mesh = MyMesh('bunny.off')
mesh.compute_laplacian()

The `self` parameter actually defined the instance of the class `MyMesh` on which the method is called.
Equivalentely, one could run `MyMesh.compute_laplacian(mesh)` instead of `mesh.compute_laplacian()`.

# Diffusion on Surface 

## Exact Diffusion 

Perhaps, the most common use of the Laplace-Beltrami operator is to simulate heat diffusion on an arbitrary surface.

Recall that the Heat Equation is defined as
$$
\frac{\partial u}{\partial t} = \Delta u
$$

Given an initial function $u_0$, the values $u_t$ at time $t$ can be obtained by discretizing the equation using single backward Euler Step, which leads to 
$$
\left(I_d - t\Delta \right) u_t = u_0
$$

Note however that the mathematical laplacian $\Delta$ is <b>negative semidefinite</b>, while our laplacian $L$ is <b>positive semidefinite</b>, and $L\simeq -\Delta$. 

Replacing $\Delta$ by $-L$ and multiplying the left and right side of the equation by $A$, which removes the inverse $A^{-1}$, we obtain

$$
\left(A + tW \right) u_t = A u_0
$$

Therefore, given $u_0$, the value of $u$ at time $t$ can be obtained by solving this **sparse** linear system (as the matrix $A + tW$ we want to invert is sparse).

While the system is made of $n$ equations,  where $n$ is the number of vertices which can grow very large, the fact it is **sparse** enables the use of significanly faster solver than for dense systems of the same size. Note that solving the system **does not** require to actually invert $A + tW$.



## Question 4
<b> Fill the function <code>diffuse_full</code> to compute diffusion of an initial function $f$ for time $t$</b>

**Tip**: Use [`sparse.linalg.spsolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.spsolve.html)

In [15]:
def diffuse_full(f, mesh, t):
    """
    Diffuse a function f on a mesh for time t
    
    Input
    --------------
    f       : (n,) - function values
    mesh    : MyMesh - mesh on which to diffuse
    t       : float - time for which to diffuse
    
    Output
    --------------
    f_diffuse : (n,) values of f after diffusion
    """
    # TODO
    # Solve the Diffusion process using the formula above
    mesh.compute_laplacian()
    #On résoud l'équation linéaire
    f_diffuse = scipy.sparse.linalg.spsolve(mesh.A + t* (mesh.W), mesh.A @ np.array(f).T)
    
    return f_diffuse

## Question 5
<b> See what happens when diffusing a dirac function</b>

**Tip**:
<ol>
    <li> Plotting the dirac function centered on index <code>ind</code> can be done with <code>plu.plot(mesh, points=ind)</code> </li>
    <li> Plotting the diffused function <code>f_diffuse</code> can be done with <code>plu.plot(mesh, f_diffuse, colormap='Reds')</code> </li>

In [16]:
## TODO
deb=np.random.randint(0,len(mesh.vertices),3)
plu.plot(mesh,points=deb)

#dirac en 3 points aléatoires:
f_0=np.zeros(len(mesh.vertices))
f_0[deb]=1

f_diffused = diffuse_full(f_0, mesh, t=0.01)
plu.plot(mesh, f_diffused, colormap='Reds')


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

## Approximate Diffusion 

As seen in class, the spectrum of the Laplace-Beltrami operator provides a functional basis which generalizes the standard Fourier Analysis to all surfaces.

As the Laplacian is only positive **semidefinite**, its spectrum is obtained by solving the **generalized** eigenvalue problem $W\phi = \lambda A \phi$.

This spectrum consists in eigenvalues - eigenfunctions pairs $\left(\lambda_j, \phi_j\right)_j$.

In practice, we only compute the first $K$ eigenfunctions where $K$ lies in $[10,100]$. This defines an **orthogonal** basis with respect to the inner product defined by $A$.

In practice, the spectrum is stored as follows:

- The eigenvalues $ \left(\lambda_j\right)_j $ are stored in a list and seen as a diagonal matrix $\Lambda$

- The eigenvectors $\left(\phi_j\right)_j$ are stored as columns of matrix $\Phi$

and satisfies the following properties:
- <b>Orthogonality</b> of the inner product defined by $A$: $\Phi^\top A \Phi = I_K$. This is the discrete translation of $\langle \phi_i, \phi_j\rangle_{A} = \delta_{ij}$, where $\delta$ is here the Kronecker symbol
- <b>Eigendecomposition</b>: $L\Phi = \Phi \Lambda$ (or $W\Phi = A\Phi \Lambda$). This is the discrete translation of $L\phi_i=\lambda_i\phi_i$
- The <b>projection</b> $\alpha\in\mathbb{R}^K$ of a function $f\in\mathbb{R}^n$ on the basis is: $\alpha = \Phi^\top A f$. This is the translation of $\alpha_i=\langle\phi_i, f\rangle_A$
- Coefficients $\alpha\in\mathbb{R}^K$ in the basis define a function $f\in\mathbb{R}^n$ on the shape as: $\Phi\alpha = f$. This is the translation of $f=\sum_{i=0}^K \alpha_i\phi_i$


Let's add the eigendecomposition of the Laplacian to the mesh class as a new method.

In [17]:
class MyMesh:
    def __init__(self, path):
        """
        Initialize the mesh from a path
        """
        self.vertices, self.faces = read_off(path)
        
    def compute_laplacian(self):
        self.A = area_matrix(self.vertices, self.faces)
        self.W = cotan_matrix(self.vertices, self.faces)
        
    def compute_eigendecomposition(self, K):
        self.eigenvalues, self.eigenvectors = sparse.linalg.eigsh(self.W, M=self.A,
                                                                  k=K, sigma=-0.01)
        # The sigma parameter allows to stabilize the eigendecomposition
        

In [18]:
mesh = MyMesh("bunny.off")
mesh.compute_laplacian()
mesh.compute_eigendecomposition(100)  ## This can take some time

## Question 6
<ol>
    <li><b> Visualize some eigenfunctios of the Laplacian.</b></li>
    <li><b> Visualize the effect of projecting a dirac function on a basis of different size</b>. Use the above explanations to project</li>
</ol>

**Tip** To plot a general function, it is good to use a simple colormap like `"coolwarm"`: <code>plu.plot(mesh, function, colormap="coolwarm")</code>

In [33]:
plu.plot(mesh, mesh.eigenvectors[:, 1], colormap='coolwarm') #first eigenfunction

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

In [31]:
## TODO Plot projected dirac
#Dirac :
deb=np.random.randint(0,len(mesh.vertices),3)
f_0=np.zeros(len(mesh.vertices))
f_0[deb]=1

alpha = mesh.eigenvectors.T @ (mesh.A @ f_0) #alpha_i = <phi_i, f>

f_projected = mesh.eigenvectors @ alpha
plu.plot(mesh,points=deb)
plu.plot(mesh, f_projected, colormap='coolwarm')



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

Coming back to Diffusion, we wish to solve the Diffusion process using spectral analysis. Recall the discretized diffusion equation was
$$
\left(A + tW \right) u_t = A u_0
$$

If we consider $u_t$ to lie in a basis of size $K$ - that is $u_t =\Phi\alpha_t$, multiplying by $\Phi^\top$ on the left gives
$$
\left(I_K + t\Lambda \right) \alpha_t = \Phi^\top A u_0
$$

Since the leftmost term is diagonal, this means $u_t =\Phi\alpha_t$ with the element-wise value:
$$
(\alpha_t)_j = \frac{\beta_j}{1+\lambda_j}
$$

where $\beta=\Phi^\top A u_0$ is actually the **projection** of $u_0$ in the basis.

Note that <b>this only requires elementary matrix multiplications</b> and not solving a linear system.

**Note**: In practice we can have a better approximation by coming back to the original heat equation $\frac{\partial u_t}{\partial t} = \Delta u_t$.

Using $u_t =\Phi\alpha_t$ and $\Delta\Phi=-\Phi\Lambda$, we can obtain $\frac{\partial \alpha_t}{\partial t} = -\Lambda \alpha_t$, with $\Lambda$ being a diagonal matrix. We then obtain
$$
(\alpha_t)_j = \exp(-\lambda_j t) \beta_j
$$

## Question 7
<b> Fill the function <code>diffuse_spectral</code> which diffuses a function f for a time t</b>


In [21]:
def diffuse_spectral(f, mesh, t, k):
    """
    Diffuse a function f on a mesh for time t
    
    Input
    --------------
    f       : (n,) - function values
    mesh    : MyMesh - mesh on which to diffuse
    t       : float - time for which to diffuse
    k       : int - size of the basis to use for diffusion
    
    Output
    --------------
    f_diffuse : (n,) values of f after diffusion
    """
    # TODO
    # Solve the Diffusion process using spectral analysis.
    # Use the formula above.
    # Note that you should return the function u_t, NOT alpha_t !
    mesh.compute_eigendecomposition(k)

    beta = mesh.eigenvectors.T @ mesh.A @ f
    #on calcul directement en utilisant la formule de dessus en écrivant directement l'expression de beta en rajoutan le@phi avant pour bien revenir à f (et non alpha)
    # (bonne dimension : n*k @ k*k @ k*n @ n*n @ n*1 = k*1)
    
    f_diffuse = mesh.eigenvectors @ (np.exp(-t*mesh.eigenvalues) * beta)

    
    return f_diffuse


## Question 8
<b>Compare the results of the diffusion using the exact and spectral approach on :</b>

- a dirac function.
- a function $\Phi\alpha$ with $\alpha\sim \otimes^k\mathcal{U}(0,1)$ 

Note that similarly to standard Fourier analysis, a dirac function require the entire range of frequencies to be represented in the basis.

In [22]:
## TODO

# Dirac :
deb=np.random.randint(0,len(mesh.vertices),2)
f_0=np.zeros(len(mesh.vertices))
f_0[deb]=1

#exact
f_diffused = diffuse_full(f_0, mesh, t=0.004)


f_projected0 = diffuse_spectral(f_0, mesh, t=0.004, k=100)
#f_projected = diffuse_spectral(f_0, mesh, t=0.001, k=100)
#f_projected1 = diffuse_spectral(f_0, mesh, t=0.01, k=100)
#f_projected2 = diffuse_spectral(f_0, mesh, t=0.04, k=100)

plu.plot(mesh,points=deb)

plu.plot(mesh, f_diffused, colormap='Reds')

plu.plot(mesh, f_projected0, colormap='Reds')
#plu.plot(mesh, f_projected, colormap='Reds')
#plu.plot(mesh, f_projected1, colormap='Reds')
#plu.plot(mesh, f_projected2, colormap='Reds')




Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

In [23]:
#Géneration uniforme * phi :
k=100
alpha = np.random.rand(k)
f_0 = mesh.eigenvectors @ alpha

#exact
f_diffused = diffuse_full(f_0, mesh, t=0.004)

f_projected0 = diffuse_spectral(f_0, mesh, t=0.004, k=100)
#f_projected = diffuse_spectral(f_0, mesh, t=0.001, k=100)
#f_projected1 = diffuse_spectral(f_0, mesh, t=0.01, k=100)
#f_projected2 = diffuse_spectral(f_0, mesh, t=0.04, k=100)

plu.plot(mesh, f_diffused, colormap='Reds')

plu.plot(mesh, f_projected0, colormap='Reds')
#plu.plot(mesh, f_projected, colormap='Reds')
#plu.plot(mesh, f_projected1, colormap='Reds')
#plu.plot(mesh, f_projected2, colormap='Reds')


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

On observe des légères differences à des t=0 (intensité plus concentré dans le cas non exact), à un temps plus long, le cas approximation se concentre de plus beaucoup plus sur certaines parties du lapin (oreilles...) que le exact. Cependant le cas avec approximation est bien plus rapide.

# The Heat Kernel Signature

The [Heat Kernel Signature](http://www.lix.polytechnique.fr/~maks/papers/hks.pdf) (**HKS**) is a pointwise descriptor which provides local information invariant under isometry. In particular, this allows to compute correspondences between near-isometric surfaces.

Given the spectrum $\left(\lambda_j, \phi_j\right)$ of the Laplacian of a shape, the HKS is defined as
$$
HKS(x,t) = C_t\sum_j e^{-\lambda_j t} \phi_j(x)^2
$$

with $C_t^{-1} = \sum_j  e^{-\lambda_j t}$

## Question 9
<b>Fill the `compute_HKS` function to compute HKS descriptor for some time parameters $t$</b>

In [24]:
def compute_HKS(mesh, n_times, k):
    # Defines a list of time parameters at which to compute HKS
    abs_ev = sorted(np.abs(mesh.eigenvalues[:k]))
    t_list = np.geomspace(4*np.log(10)/abs_ev[-1], 4*np.log(10)/abs_ev[1], n_times)  # (n_times,)
    
    ## TODO COMPUTE HKS
    HKS = np.zeros((len(mesh.vertices), n_times))
    for i in range(n_times):
        t = t_list[i]
        HKS[:, i] = np.sum(np.exp(-t * mesh.eigenvalues[:k]) * (mesh.eigenvectors[:, :k] ** 2), axis=1)

    return HKS

HKS can later be used as a means to compute shape correspondence. Given two shapes $\mathcal{X}$ and $\mathcal{Y}$, we define the HKS distance between $x\in\mathcal{X}$ and $y\in\mathcal{Y}$ as
$$
d_{HKS}(x,y) = \int_t |HKS_\mathcal{X}(x,t) -  HKS_\mathcal{Y}(y,t)|d\log t
$$

The logarithmic sampling in `compute_HKS` naturally amounts for the $d\log t$ integration, so two embeddings obtained with our function `compute_HKS` can simply be compared using the standard norm of the difference.

Eventually, correspondences $T_{\mathcal{Y}\mathcal{X}}$ from $\mathcal{Y}$ to $\mathcal{X}$ can be obtained by setting
$$
T_{\mathcal{Y}\mathcal{X}}(y) = \underset{y\in\mathcal{Y}}{\text{argmin}}\ d_{HKS}(x,y)
$$

## Question 10
<b>Use HKS to compute correspondences between two surfaces</b>

**Tips**: 

- $T_{\mathcal{Y}\mathcal{X}}$ is stored as a $n_\mathcal{Y}$ dimensional array where the $i$-th entry gives the index of the image of $y_i$ in $\mathcal{X}$ </li>
- <b>Warning:</b> Be careful not to overflow the memory </li>
- Plot correspondences $T_{\mathcal{Y}\mathcal{X}}$ using <code>plu.plot_p2p(mesh1, mesh2, T_YX)</code></li>


In [25]:
mesh1 = MyMesh("hand1.off")
mesh2 = MyMesh("hand2.off")

mesh1.compute_laplacian()
mesh1.compute_eigendecomposition(100)
mesh2.compute_laplacian()
mesh2.compute_eigendecomposition(100)
print('end')

end


In [26]:

T_YX = np.zeros(len(mesh2.vertices), dtype=int)  # (n_Y,)
HKS1 = compute_HKS(mesh1, n_times=50, k=50)
HKS2 = compute_HKS(mesh2, n_times=50, k=50)

for i in range(len(mesh2.vertices)):
    if i % 1000 == 0:
        print(f"Processing vertex {i}/{len(mesh2.vertices)}")
        
    distances = np.linalg.norm(HKS1 - HKS2[i, :], axis=1) #calcul de la distance(ressemblance) entre le ième sommet de Y et tous les sommets de X
    T_YX[i] = np.argmin(distances) #on prend l'indice du plus proche(resssemblant) du ième sommet de Y dans X




Processing vertex 0/26000
Processing vertex 1000/26000
Processing vertex 2000/26000
Processing vertex 3000/26000
Processing vertex 4000/26000
Processing vertex 5000/26000
Processing vertex 6000/26000
Processing vertex 7000/26000
Processing vertex 8000/26000
Processing vertex 9000/26000
Processing vertex 10000/26000
Processing vertex 11000/26000
Processing vertex 12000/26000
Processing vertex 13000/26000
Processing vertex 14000/26000
Processing vertex 15000/26000
Processing vertex 16000/26000
Processing vertex 17000/26000
Processing vertex 18000/26000
Processing vertex 19000/26000
Processing vertex 20000/26000
Processing vertex 21000/26000
Processing vertex 22000/26000
Processing vertex 23000/26000
Processing vertex 24000/26000
Processing vertex 25000/26000


In [27]:
plu.plot_p2p(mesh1, mesh2, T_YX)

# Heat Method for geodesic (BONUS) 

The [Heat method](https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/paperCACM.pdf) uses the diffusion process of a dirac function to compute geodesic distances across a mesh. One advantage of this method is speed and a formulation independant from the specific triangulation of the mesh as long as differential operators are defined (**results are not though**).

Given a point $x$, we start from a diract function $\delta_x$ centered on $x$. The heat method is solved as follows:

<ol>
    <li> Integrate the heat equation with $u_0=\delta_x$, that is solve $\left(A + tW \right) u_t = A \delta_x$. This diffuses a lot heat to points close to $x$ and the value decreases as we for further away points. </li>
    <li> Evaluate the vector field $X=-\nabla u_t / \|\nabla u_t \|$ which gives the (opposite) normalized direction of diffusion, hopefully going *geodesically* towards $x$ </li>
    <li> Solve the Poisson Equation $\Delta \varphi = \nabla\cdot X$ to obtain the geodesic distance $\varphi$ ensuring the distance to $x$ is $0$ by adding the necessary constant</li>
</ol>

The time of diffusion $t$ is usually set as $h^2$ with $h$ the average edge length of the mesh.

## Question 11 (Bonus)
**Implement the heat method usng the gradient and divergence operator we provide**

In [28]:
def grad_f(f, vertices, faces, normals):
    """
    Compute the gradient of a function on a mesh

    Parameters
    --------------------------
    f          : (n,) function value on each vertex
    vertices   : (n,3) coordinates of vertices
    faces      : (m,3) indices of vertices for each face
    normals    : (m,3) normals coordinate for each face
    face_area : (m,) - Optional, array of per-face area, for faster computation
    use_sym    : bool - If true, uses the (slower but) symmetric expression
                 of the gradient

    Output
    --------------------------
    gradient : (m,3) gradient of f on the mesh
    """
    v1 = vertices[faces[:,0]]  # (m,3)
    v2 = vertices[faces[:,1]]  # (m,3)
    v3 = vertices[faces[:,2]]  # (m,3)

    f1 = f[faces[:,0]]  # (m,)
    f2 = f[faces[:,1]]  # (m,)
    f3 = f[faces[:,2]]  # (m,)

    # Compute area for each face
    face_areas = 0.5 * np.linalg.norm(np.cross(v2-v1,v3-v1),axis=1)  # (m)

   

    grad1 = np.cross(normals, v3-v2)/(2*face_areas[:,None])  # (m,3)
    grad2 = np.cross(normals, v1-v3)/(2*face_areas[:,None])  # (m,3)
    grad3 = np.cross(normals, v2-v1)/(2*face_areas[:,None])  # (m,3)

    gradient = f1[:,None] * grad1 + f2[:,None] * grad2 + f3[:,None] * grad3

    return gradient


def div_f(f, vertices, faces, normals, vert_areas):
    """
    Compute the divergence of a vector field on a mesh

    Parameters
    --------------------------
    f          : (m,3) vector field on each face
    vertices   : (n,3) coordinates of vertices
    faces      : (m,3) indices of vertices for each face
    normals    : (m,3) normals coordinate for each face
    vert_area : (m,) - array of per-vertex area

    Output
    --------------------------
    divergence : (n,) divergence of f on the mesh
    """
    n_vertices = vertices.shape[0]

    v1 = vertices[faces[:,0]]  # (m,3)
    v2 = vertices[faces[:,1]]  # (m,3)
    v3 = vertices[faces[:,2]]  # (m,3)

    grad1 = np.einsum('ij,ij->i', np.cross(normals, v3 - v2) / 2, f)  # (m,)
    grad2 = np.einsum('ij,ij->i', np.cross(normals, v1 - v3) / 2, f)  # (m,)
    grad3 = np.einsum('ij,ij->i', np.cross(normals, v2 - v1) / 2, f)  # (m,)

    I = np.concatenate([faces[:, 0], faces[:, 1], faces[:, 2]])  # (3*m)
    J = np.zeros_like(I)
    V = np.concatenate([grad1, grad2, grad3])

    div_val = sparse.coo_matrix((V, (I, J)), shape=(n_vertices, 1)).todense()

    return np.asarray(div_val).flatten() / vert_areas